In [1]:
import os
import sys
import time
import pandas as pd
import MySQLdb as mysql
import matplotlib.pyplot as plt

In [2]:
print(sys.version)

3.6.5 |Anaconda custom (64-bit)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [3]:
#JSON credentials
login = pd.read_json(os.getcwd() + "/db_credentials.json", typ="series").to_dict()

#db connection
conn = mysql.connect(host=login["host"],
                     user=login["user"],
                     passwd=login["passwd"],
                     db=login["db"])    

#db cursor
cur = conn.cursor()

In [4]:
def result_set(query):
    """ Function for performing MySQL queries.
    ----- Args:
            Takes input str query.
    ----- Returns:
            sql result set as a pandas.DataFrame
    """
    return pd.read_sql(query, con=conn)

In [5]:
#NUMBER OF POSTS IN THE DATABASE
result_set('SELECT MAX(id) AS ROWS FROM forum_data')

,ROWS
0,40623785


### Changing Datetime Column

In [6]:
#cur.execute("UPDATE forum_data SET datetime = STR_TO_DATE(datetime,'%m/%d/%Y %h:%i %p');")
#conn.commit()

### Export Player Data

In [7]:
result_set('SELECT * FROM forum_data LIMIT 10')

,id,name,lvlRaceClass,achievements,postCount,datetime,postText
0,1,Ornyx,NaN,NaN,1282,2016-09-01 00:02:00,"Heroes of the Horde and the Alliance, Azeroth ..."
1,2,Blakkwydow,110 Blood Elf Warrior,22360,14,2017-11-04 15:00:00,This has been an ongoing and increasingly wors...
2,3,Perl,14 Blood Elf Priest,0,31844,2017-11-04 15:03:00,"If it's a bug, the best place to report it is ..."
3,4,Healsprocket,110 Gnome Priest,19450,70,2017-11-04 17:34:00,Have been trying to connect for about half an ...
4,5,Kuddlebugg,106 Gnome Mage,8295,3753,2017-11-04 15:04:00,What happens when you try? Do you disconnect ...
5,6,Blakkwydow,110 Blood Elf Warrior,22360,14,2017-11-04 15:49:00,"I disconnect after a very long loading screen,..."
6,7,Kozzae,110 Troll Rogue,20750,18653,2017-11-04 17:40:00,You will want to post in the tech support foru...
7,8,Healsprocket,110 Gnome Priest,19450,70,2017-11-04 17:48:00,Thanks. I went ahead and posted under someone ...
8,9,Azevara,110 Blood Elf Priest,18175,9641,2017-11-05 08:01:00,Double up the digits and double down on fun.Pl...
9,10,Regg,110 Blood Elf Rogue,15300,13920,2017-11-05 08:02:00,Execute Lounge 66. /Palpatine


In [8]:
t1 = time.time()

results = result_set("""\
SELECT name, lvlRaceClass AS class, achievements, MIN(datetime) AS min_date, MAX(datetime) AS max_date 
FROM forum_data
GROUP BY name, lvlRaceClass, achievements
""")

t2 = time.time()

print("Runtime: %.1f seconds." % (t2-t1))

Runtime: 674.9 seconds.


In [10]:
results.to_pickle(os.getcwd() + '/wow_player_stats.pkl')